In [1]:
import re

from utils_v2_for_jupyter import *
from nucleotide_toolkit import *

from features import *
import pandas as pd
from icecream import ic
from collections import defaultdict


# importing stuff
sequence = parse_fasta_file(
    "../data/sequences/tp53/Homo_sapiens_ENST00000420246_2_sequence.fa")
targetscan_df = pd.read_csv(
    "../data/supplementary_files/targetscan.tsv", sep="\t")

sequence = sequence[:1000]

In [2]:

clash = import_clash_df()


In [2]:
def align_sequences(seq1, seq2, allow_wobbles=False):


    alignment_string = []

    wobble_count = 0

    for nucleotide_1, nucleotide_2 in zip(seq1, seq2):
        
        if nucleotide_1 == nucleotide_2:
            alignment_string.append("1")

        elif allow_wobbles and ((nucleotide_1 == "G" and nucleotide_2 == "U") or (nucleotide_1 == "U" and nucleotide_2 == "G")):
            alignment_string.append("2")
            
            wobble_count += 1
        else:
            alignment_string.append("0")

    return "".join(alignment_string), wobble_count


In [16]:
import numpy as np

def align_sequences_vect(seq1, seq2, allow_wobbles=False):
    seq1_array = np.array(list(seq1))
    seq2_array = np.array(list(seq2))
    equal_mask = seq1_array == seq2_array
    equal_masked = equal_mask.astype(int)
    
    wobble_mask = allow_wobbles & ((seq1_array == 'G') & (seq2_array == 'U') | (seq1_array == 'U') & (seq2_array == 'G'))
    
    wobble_masked = wobble_mask.astype(int) * 2
    
    alignment_array = equal_masked + wobble_masked
    
    return ''.join(alignment_array.astype(str)), ''.join(wobble_masked.astype(str))

seq1 = "AACUAUACAACCUACUACCUCA"
seq2 = "GAGCAUGAAAAUGGUUCUAUGA"

asd = align_sequences(seq1, seq2,allow_wobbles=True)
print(asd)

('0100110011002001000101', '0000000000002000000000')


In [17]:
sequence = parse_fasta_file(
    "../data/sequences/tp53/Homo_sapiens_ENST00000420246_2_sequence.fa")

targetscan_df = pd.read_csv(
    "../data/supplementary_files/targetscan.tsv", sep="\t")

In [30]:
import cProfile

def find_matches_pro(sequence, mirna_df, allow_wobbles=False, minimum_matches=7):
    def inner():
        # Preparing stuff
        df_names = mirna_df["name"].tolist()

        df_sequences = mirna_df["sequence"].tolist()

        names, starts, mirna_sequences, mrna_sequences, alignment_strings, wobble_strings = [], [], [], [], [], []

        # For each miRNA
        for i, mirna_sequence in enumerate(df_sequences):
            # Create a generator for sliding windows
            generator = sliding_window(
                thymine_to_uracil(sequence), len(mirna_sequence))


            alignment_string, wobble_string = align_sequences_vect(generator, mirna_sequence, allow_wobbles=allow_wobbles)

            # Add the results to the lists
            names.append(df_names[i])

            mirna_sequences.append(mirna_sequence)

            alignment_strings.append(alignment_string)
            wobble_strings.append(wobble_string)


        df = pd.DataFrame(
            {
                "name": names,

                "mirna_sequence": mirna_sequences,
                "mrna_sequence": mrna_sequences,
                "alignment_string": alignment_strings,
                "wobble_string": wobble_strings
            }
        )

        # adding seed column
        df["seed"] = df["mirna_sequence"].str[-8:-1]

        return df

    profiler = cProfile.Profile()
    profiler.enable()
    result = inner()
    profiler.disable()
    profiler.print_stats(sort='cumulative')

    return result

In [31]:
df = find_matches_pro(sequence[:1000], targetscan_df, allow_wobbles=True, minimum_matches=7)

/tmp/ipykernel_32238/1372137472.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  equal_mask = seq1_array == seq2_array


AttributeError: 'bool' object has no attribute 'astype'

In [67]:
import pandas as pd

def slide_and_compare_vect_df(short_seq, long_seq, n):
    # Convert the short and long sequences to NumPy arrays of characters
    short_array = np.array(list(short_seq))
    long_array = np.array(list(long_seq))

    # Create a sliding window view of the long array using NumPy's stride_tricks module
    window_array = np.lib.stride_tricks.sliding_window_view(long_array, len(short_seq))

    # Compare each window against the short array using NumPy's broadcasting and element-wise boolean comparison
    matches = np.sum(window_array == short_array, axis=1)

    # Generate an array of indices for the starting positions of each window in the long sequence
    indices = np.arange(len(long_seq) - len(short_seq) + 1)

    # Generate an array of alignment strings for each window
    alignments = ["".join(["1" if i == 1 else "0" for i in row]) for row in (window_array == short_array)]

    # Create a pandas DataFrame with the indices, alignments, and match counts
    df = pd.DataFrame({'Index': indices, 'Alignment': alignments, 'Matches': matches})

    # Filter the DataFrame to include only windows with at least n matches
    df_filtered = df[df['Matches'] >= n]

    # Return the filtered DataFrame
    return df_filtered




short = "AACUAUACAACCUACUACCUCA"

asd = slide_and_compare_vect_df("AACUAUACAACCUACUACCUCA", sequence, 7)

In [68]:
asd

,Index,Alignment,Matches
3,3,0100110011000001000101,8
5,5,0001001010001011100010,8
8,8,0000100000011100110100,7
11,11,1101000000000111000010,7
12,12,1100010101000001001000,7
...,...,...,...
1456,1456,0110111011101000001000,10
1457,1457,1000001010001011011010,9
1460,1460,1000100000011010111100,9
1464,1464,1111010100110011000100,11


In [65]:
clash

,seq_ID,microRNA_name,miRNA_start,miRNA_end,miRNA_seq,mRNA_name,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,...,folding_energy,5'UTR,CDS,3'UTR,folding_class,conservation_score,log2_target_enrichment,CLASH_single_reads_ovlp,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp
0,0727A-1038930_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000113328_ENST00000340828_CCNG1_mRNA,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,...,-25.1,NaN,NaN,1.0,III,0.210342,-0.020802,270.0,NaN,NaN
1,L1HS-1112536_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000100697_ENST00000343455_DICER1_mRNA,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,...,-24.4,NaN,1.0,NaN,II,NaN,0.628759,24.0,NaN,1.0
2,L2HS-818542_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000080546_ENST00000436639_SESN1_mRNA,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,...,-22.2,NaN,1.0,1.0,III,NaN,0.022816,56.0,1.0,1.0
3,L2HS-1161339_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000164190_ENST00000282516_NIPBL_mRNA,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,...,-22.1,NaN,1.0,NaN,III,NaN,-0.007294,7.0,NaN,NaN
4,L2-407944_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000138785_ENST00000340139_INTS12_mRNA,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,...,-21.9,NaN,1.0,NaN,III,NaN,0.026476,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,L2HS-3158593_1,MIMAT0018349_MirBase_miR-3934_microRNA,1,22,TCAGGTGTGGAAACTGAGGCAG,ENSG00000198712_ENST00000361739_MT-CO2_mRNA,199,253,ATCCTGCCCGCCATCATCCTAGTCCTCATCGCCCTCCCATCCCTAC...,1,...,-13.5,NaN,1.0,NaN,V,NaN,NaN,593.0,NaN,NaN
18510,L2HS-578047_3,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000140988_ENST00000343262_RPS2_mRNA,345,388,CTTCCTGGGGGCCTCTCTCAAGGATGAGGTTTTGAAGATTATGC,4,...,-18.8,NaN,1.0,NaN,I,NaN,NaN,662.0,NaN,NaN
18511,L1HS-550121_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000080824_ENST00000216281_HSP90AA1_mRNA,666,720,AGTACGCTTGGGAGTCCTCAGCAGGGGGATCATTCACAGTGAGGAC...,2,...,-18.5,NaN,1.0,NaN,I,NaN,NaN,178.0,NaN,NaN
18512,L2HS-896514_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000135404_ENST00000257857_CD63_mRNA,614,657,TAAGTGCTGTGGGGCTGCTAACTACACAGATTGGGAGAAAATCC,2,...,-17.9,NaN,1.0,NaN,III,NaN,NaN,47.0,NaN,NaN


In [66]:
clash["new"] = clash.apply(slide_and_compare_vect_df, args=("miRNA_seq", "mRNA_seq_extended", 7), axis=1)

TypeError: slide_and_compare_vect_df() missing 3 required positional arguments: 'long_seq', 'longcolname', and 'n'

In [58]:
clash

,seq_ID,microRNA_name,miRNA_start,miRNA_end,miRNA_seq,mRNA_name,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,...,folding_energy,5'UTR,CDS,3'UTR,folding_class,conservation_score,log2_target_enrichment,CLASH_single_reads_ovlp,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp
0,0727A-1038930_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000113328_ENST00000340828_CCNG1_mRNA,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,...,-25.1,NaN,NaN,1.0,III,0.210342,-0.020802,270.0,NaN,NaN
1,L1HS-1112536_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000100697_ENST00000343455_DICER1_mRNA,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,...,-24.4,NaN,1.0,NaN,II,NaN,0.628759,24.0,NaN,1.0
2,L2HS-818542_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000080546_ENST00000436639_SESN1_mRNA,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,...,-22.2,NaN,1.0,1.0,III,NaN,0.022816,56.0,1.0,1.0
3,L2HS-1161339_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000164190_ENST00000282516_NIPBL_mRNA,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,...,-22.1,NaN,1.0,NaN,III,NaN,-0.007294,7.0,NaN,NaN
4,L2-407944_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000138785_ENST00000340139_INTS12_mRNA,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,...,-21.9,NaN,1.0,NaN,III,NaN,0.026476,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,L2HS-3158593_1,MIMAT0018349_MirBase_miR-3934_microRNA,1,22,TCAGGTGTGGAAACTGAGGCAG,ENSG00000198712_ENST00000361739_MT-CO2_mRNA,199,253,ATCCTGCCCGCCATCATCCTAGTCCTCATCGCCCTCCCATCCCTAC...,1,...,-13.5,NaN,1.0,NaN,V,NaN,NaN,593.0,NaN,NaN
18510,L2HS-578047_3,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000140988_ENST00000343262_RPS2_mRNA,345,388,CTTCCTGGGGGCCTCTCTCAAGGATGAGGTTTTGAAGATTATGC,4,...,-18.8,NaN,1.0,NaN,I,NaN,NaN,662.0,NaN,NaN
18511,L1HS-550121_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000080824_ENST00000216281_HSP90AA1_mRNA,666,720,AGTACGCTTGGGAGTCCTCAGCAGGGGGATCATTCACAGTGAGGAC...,2,...,-18.5,NaN,1.0,NaN,I,NaN,NaN,178.0,NaN,NaN
18512,L2HS-896514_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000135404_ENST00000257857_CD63_mRNA,614,657,TAAGTGCTGTGGGGCTGCTAACTACACAGATTGGGAGAAAATCC,2,...,-17.9,NaN,1.0,NaN,III,NaN,NaN,47.0,NaN,NaN


In [57]:
def find_matches_from_clash(clash_df):
    
    

,Index,Alignment,Matches
0,3,0100110011000001000101,8
1,5,0001001010001011100010,8
2,8,0000100000011100110100,7
3,11,1101000000000111000010,7
4,12,1100010101000001001000,7
...,...,...,...
434,1456,0110111011101000001000,10
435,1457,1000001010001011011010,9
436,1460,1000100000011010111100,9
437,1464,1111010100110011000100,11


In [100]:
clash_df = pd.read_csv("../data/supplementary_files/clash.tsv", sep="\t")

columns_to_keep = ["microRNA_name", "miRNA_seq", "mRNA_name", "mRNA_start", "mRNA_end_extended", "mRNA_seq_extended", "seed_type"]
clash_df = clash_df[columns_to_keep]

# Split the column into new columns based on "_"
new_cols = clash_df['microRNA_name'].str.split('_', expand=True)

# Rename the new columns
new_cols.columns = ['accession', "from", 'mirna_name', 'temp']

# Concatenate the new columns to the original DataFrame
clash_df = pd.concat([clash_df, new_cols], axis=1)

# Drop the original column containing strings with 2 "_" in them
clash_df = clash_df.drop('microRNA_name', axis=1)
clash_df = clash_df.drop('temp', axis=1)
clash_df = clash_df.drop('from', axis=1)

# process mRNA name column
# Split the column into new columns based on "_"
new_cols = clash_df['mRNA_name'].str.split('_', expand=True)

# Rename the new columns
new_cols.columns = ['ensg', "enst", 'gene_name', 'temp']

# Concatenate the new columns to the original DataFrame
clash_df = pd.concat([clash_df, new_cols], axis=1)

# Drop the original column containing strings with 2 "_" in them
clash_df = clash_df.drop('mRNA_name', axis=1)
clash_df = clash_df.drop('temp', axis=1)






In [101]:
clash_df.head()

,miRNA_seq,mRNA_start,mRNA_end_extended,mRNA_seq_extended,seed_type,accession,mirna_name,ensg,enst,gene_name
0,TGAGGTAGTAGGTTGTATAGTT,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,noncanonical_seed,MIMAT0000062,let-7a,ENSG00000113328,ENST00000340828,CCNG1
1,TGAGGTAGTAGGTTGTATAGTT,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9-mer,MIMAT0000062,let-7a,ENSG00000100697,ENST00000343455,DICER1
2,TGAGGTAGTAGGTTGTATAGTT,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,noncanonical_seed,MIMAT0000062,let-7a,ENSG00000080546,ENST00000436639,SESN1
3,TGAGGTAGTAGGTTGTATAGTT,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,noncanonical_seed,MIMAT0000062,let-7a,ENSG00000164190,ENST00000282516,NIPBL
4,TGAGGTAGTAGGTTGTATAGTT,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,noncanonical_seed,MIMAT0000062,let-7a,ENSG00000138785,ENST00000340139,INTS12


NameError: name 'import_clash_df' is not defined

In [112]:
clash

,seq_ID,miRNA_start,miRNA_end,miRNA_seq,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,experiments,experiments_list,...,conservation_score,log2_target_enrichment,CLASH_single_reads_ovlp,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp,accession,mirna_name,ensg,enst,gene_name
0,0727A-1038930_1,1,22,TGAGGTAGTAGGTTGTATAGTT,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,3,"E2,E3,E4",...,0.210342,-0.020802,270.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000113328,ENST00000340828,CCNG1
1,L1HS-1112536_1,1,22,TGAGGTAGTAGGTTGTATAGTT,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,2,"E3,E4",...,NaN,0.628759,24.0,NaN,1.0,MIMAT0000062,let-7a,ENSG00000100697,ENST00000343455,DICER1
2,L2HS-818542_2,1,22,TGAGGTAGTAGGTTGTATAGTT,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,1,E4,...,NaN,0.022816,56.0,1.0,1.0,MIMAT0000062,let-7a,ENSG00000080546,ENST00000436639,SESN1
3,L2HS-1161339_2,1,22,TGAGGTAGTAGGTTGTATAGTT,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,1,E4,...,NaN,-0.007294,7.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000164190,ENST00000282516,NIPBL
4,L2-407944_2,1,22,TGAGGTAGTAGGTTGTATAGTT,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,1,E4,...,NaN,0.026476,6.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000138785,ENST00000340139,INTS12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,L2HS-3158593_1,1,22,TCAGGTGTGGAAACTGAGGCAG,199,253,ATCCTGCCCGCCATCATCCTAGTCCTCATCGCCCTCCCATCCCTAC...,1,1,E4,...,NaN,NaN,593.0,NaN,NaN,MIMAT0018349,miR-3934,ENSG00000198712,ENST00000361739,MT-CO2
18510,L2HS-578047_3,1,23,TAGCCCCCAGGCTTCACTTGGCG,345,388,CTTCCTGGGGGCCTCTCTCAAGGATGAGGTTTTGAAGATTATGC,4,1,E4,...,NaN,NaN,662.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000140988,ENST00000343262,RPS2
18511,L1HS-550121_2,1,23,TAGCCCCCAGGCTTCACTTGGCG,666,720,AGTACGCTTGGGAGTCCTCAGCAGGGGGATCATTCACAGTGAGGAC...,2,1,E3,...,NaN,NaN,178.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000080824,ENST00000216281,HSP90AA1
18512,L2HS-896514_2,1,23,TAGCCCCCAGGCTTCACTTGGCG,614,657,TAAGTGCTGTGGGGCTGCTAACTACACAGATTGGGAGAAAATCC,2,1,E4,...,NaN,NaN,47.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000135404,ENST00000257857,CD63


In [107]:
clash

,seq_ID,miRNA_start,miRNA_end,miRNA_seq,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,experiments,experiments_list,...,conservation_score,log2_target_enrichment,CLASH_single_reads_ovlp,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp,accession,mirna_name,ensg,enst,gene_name
0,0727A-1038930_1,1,22,TGAGGTAGTAGGTTGTATAGTT,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,3,"E2,E3,E4",...,0.210342,-0.020802,270.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000113328,ENST00000340828,CCNG1
1,L1HS-1112536_1,1,22,TGAGGTAGTAGGTTGTATAGTT,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,2,"E3,E4",...,NaN,0.628759,24.0,NaN,1.0,MIMAT0000062,let-7a,ENSG00000100697,ENST00000343455,DICER1
2,L2HS-818542_2,1,22,TGAGGTAGTAGGTTGTATAGTT,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,1,E4,...,NaN,0.022816,56.0,1.0,1.0,MIMAT0000062,let-7a,ENSG00000080546,ENST00000436639,SESN1
3,L2HS-1161339_2,1,22,TGAGGTAGTAGGTTGTATAGTT,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,1,E4,...,NaN,-0.007294,7.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000164190,ENST00000282516,NIPBL
4,L2-407944_2,1,22,TGAGGTAGTAGGTTGTATAGTT,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,1,E4,...,NaN,0.026476,6.0,NaN,NaN,MIMAT0000062,let-7a,ENSG00000138785,ENST00000340139,INTS12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,L2HS-3158593_1,1,22,TCAGGTGTGGAAACTGAGGCAG,199,253,ATCCTGCCCGCCATCATCCTAGTCCTCATCGCCCTCCCATCCCTAC...,1,1,E4,...,NaN,NaN,593.0,NaN,NaN,MIMAT0018349,miR-3934,ENSG00000198712,ENST00000361739,MT-CO2
18510,L2HS-578047_3,1,23,TAGCCCCCAGGCTTCACTTGGCG,345,388,CTTCCTGGGGGCCTCTCTCAAGGATGAGGTTTTGAAGATTATGC,4,1,E4,...,NaN,NaN,662.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000140988,ENST00000343262,RPS2
18511,L1HS-550121_2,1,23,TAGCCCCCAGGCTTCACTTGGCG,666,720,AGTACGCTTGGGAGTCCTCAGCAGGGGGATCATTCACAGTGAGGAC...,2,1,E3,...,NaN,NaN,178.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000080824,ENST00000216281,HSP90AA1
18512,L2HS-896514_2,1,23,TAGCCCCCAGGCTTCACTTGGCG,614,657,TAAGTGCTGTGGGGCTGCTAACTACACAGATTGGGAGAAAATCC,2,1,E4,...,NaN,NaN,47.0,NaN,NaN,MIMAT0018359,miR-3943,ENSG00000135404,ENST00000257857,CD63


In [49]:
def slide_and_compare_vect_df(df, short_col, long_col, n):
    # Convert the short and long sequences to NumPy arrays of characters
    short_array = df[short_col].values
    long_array = df[long_col].values

    # Create a sliding window view of the long array using NumPy's stride_tricks module
    window_array = np.lib.stride_tricks.sliding_window_view(long_array, len(short_array))

    # Compare each window against the short array using NumPy's broadcasting and element-wise boolean comparison
    matches = np.sum(window_array == short_array, axis=1)

    # Generate an array of indices for the starting positions of each window in the long sequence
    indices = np.arange(len(long_array) - len(short_array) + 1)

    # Generate an array of alignment strings for each window
    alignments = ["".join(["1" if i == 1 else "0" for i in row]) for row in (window_array == short_array)]

    # Create a pandas DataFrame with the indices, alignments, and match counts
    df_result = pd.DataFrame({'Index': indices, 'Alignment': alignments, 'Matches': matches})

    # Filter the DataFrame to include only windows with at least n matches
    df_filtered = df_result[df_result['Matches'] >= n]

    # Return the filtered DataFrame
    return df_filtered.reset_index(drop=True)


In [50]:
results = slide_and_compare_vect_df(clash, "miRNA_seq", "mRNA_seq_extended", 7)

In [51]:
results

,Index,Alignment,Matches


In [54]:
import numpy as np
import pandas as pd

# Define a function to compare a short sequence against a long sequence in a sliding window fashion
def slide_and_compare_vect(short_seq, long_seq):
    # Convert the short and long sequences to NumPy arrays of characters
    short_array = np.array(list(short_seq))
    long_array = np.array(list(long_seq))

    # Create a sliding window view of the long array using NumPy's stride_tricks module
    window_array = np.lib.stride_tricks.sliding_window_view(long_array, len(short_seq))

    # Compare each window against the short array using NumPy's broadcasting and element-wise boolean comparison
    matches = (window_array == short_array)

    # Generate a 1D array of alignment strings for each window
    alignments = np.array(["".join(["1" if i else "0" for i in row]) for row in matches])

    # Return the alignments
    return alignments



# Apply the slide_and_compare_vect function to the short and long sequences in the DataFrame
clash['alignments'] = np.vectorize(slide_and_compare_vect)(clash['miRNA_seq'], clash['mRNA_seq_extended'])

# Explode the alignments column to create one row per alignment
clash_exploded = clash.explode('alignments')

# Split the alignment strings into individual characters and convert them to integers
clash_exploded['matches'] = clash_exploded['alignments'].apply(lambda x: [int(i) for i in list(x)])

# Drop the alignments column
clash_final = clash_exploded.drop('alignments', axis=1)

# Print the final DataFrame
print(clash_final)


ValueError: window shape cannot be larger than input array shape

In [55]:
clash

,seq_ID,microRNA_name,miRNA_start,miRNA_end,miRNA_seq,mRNA_name,mRNA_start,mRNA_end_extended,mRNA_seq_extended,chimeras_decompressed,...,folding_energy,5'UTR,CDS,3'UTR,folding_class,conservation_score,log2_target_enrichment,CLASH_single_reads_ovlp,CLASH_cluster_ovlp,PAR_CLIP_cluster_ovlp
0,0727A-1038930_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000113328_ENST00000340828_CCNG1_mRNA,1791,1890,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,31,...,-25.1,NaN,NaN,1.0,III,0.210342,-0.020802,270.0,NaN,NaN
1,L1HS-1112536_1,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000100697_ENST00000343455_DICER1_mRNA,3857,3928,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,9,...,-24.4,NaN,1.0,NaN,II,NaN,0.628759,24.0,NaN,1.0
2,L2HS-818542_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000080546_ENST00000436639_SESN1_mRNA,2385,2434,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2,...,-22.2,NaN,1.0,1.0,III,NaN,0.022816,56.0,1.0,1.0
3,L2HS-1161339_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000164190_ENST00000282516_NIPBL_mRNA,6570,6623,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,2,...,-22.1,NaN,1.0,NaN,III,NaN,-0.007294,7.0,NaN,NaN
4,L2-407944_2,MIMAT0000062_MirBase_let-7a_microRNA,1,22,TGAGGTAGTAGGTTGTATAGTT,ENSG00000138785_ENST00000340139_INTS12_mRNA,1164,1208,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,2,...,-21.9,NaN,1.0,NaN,III,NaN,0.026476,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,L2HS-3158593_1,MIMAT0018349_MirBase_miR-3934_microRNA,1,22,TCAGGTGTGGAAACTGAGGCAG,ENSG00000198712_ENST00000361739_MT-CO2_mRNA,199,253,ATCCTGCCCGCCATCATCCTAGTCCTCATCGCCCTCCCATCCCTAC...,1,...,-13.5,NaN,1.0,NaN,V,NaN,NaN,593.0,NaN,NaN
18510,L2HS-578047_3,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000140988_ENST00000343262_RPS2_mRNA,345,388,CTTCCTGGGGGCCTCTCTCAAGGATGAGGTTTTGAAGATTATGC,4,...,-18.8,NaN,1.0,NaN,I,NaN,NaN,662.0,NaN,NaN
18511,L1HS-550121_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000080824_ENST00000216281_HSP90AA1_mRNA,666,720,AGTACGCTTGGGAGTCCTCAGCAGGGGGATCATTCACAGTGAGGAC...,2,...,-18.5,NaN,1.0,NaN,I,NaN,NaN,178.0,NaN,NaN
18512,L2HS-896514_2,MIMAT0018359_MirBase_miR-3943_microRNA,1,23,TAGCCCCCAGGCTTCACTTGGCG,ENSG00000135404_ENST00000257857_CD63_mRNA,614,657,TAAGTGCTGTGGGGCTGCTAACTACACAGATTGGGAGAAAATCC,2,...,-17.9,NaN,1.0,NaN,III,NaN,NaN,47.0,NaN,NaN
